<a href="https://colab.research.google.com/github/choheeee22/pytorch_study/blob/main/leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브 마운트 

In [2]:
cd /content/drive/MyDrive/Python 공부/leaf

/content/drive/MyDrive/Python 공부/leaf


In [3]:
pwd

'/content/drive/MyDrive/Python 공부/leaf'

In [4]:
ls

fmix.py  __pycache__/


In [5]:
#pip install kaggle --upgrade

In [6]:
 cd /content/drive/MyDrive/Python 공부/kaggle

/content/drive/MyDrive/Python 공부/kaggle


In [7]:
ls

cassava-leaf-disease-classification.zip  sample_submission.csv  train.csv
kaggle.json                              test_images/           train_images/
label_num_to_disease_map.json            test_tfrecords/


In [8]:
ls -1ha kaggle.json

kaggle.json


In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
#!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         54           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         86           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge       1186           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        156           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1925           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14228           False  
house-pr

In [10]:
#데이터 다운
#!kaggle competitions download -c cassava-leaf-disease-classification

In [11]:
ls

cassava-leaf-disease-classification.zip  sample_submission.csv  train.csv
kaggle.json                              test_images/           train_images/
label_num_to_disease_map.json            test_tfrecords/


In [12]:
#!unzip cassava-leaf-disease-classification.zip

In [13]:
ls

cassava-leaf-disease-classification.zip  sample_submission.csv  train.csv
kaggle.json                              test_images/           train_images/
label_num_to_disease_map.json            test_tfrecords/


In [14]:
import numpy as np 
import pandas as pd
import json
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms # 이미지 모양 바꿔주는 거. ex) crop, rotation
from torch.utils.data import Dataset, DataLoader

In [15]:
BATCH = 32
EPOCHS = 20

LR = 0.0001
IM_SIZE = 256

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
TRAIN_DIR = '/content/drive/MyDrive/Python 공부/kaggle/train_images/'
TEST_DIR = '/content/drive/MyDrive/Python 공부/kaggle/test_images/'

In [17]:
pwd

'/content/drive/MyDrive/Python 공부/kaggle'

In [18]:
labels = json.load(open("/content/drive/MyDrive/Python 공부/kaggle/label_num_to_disease_map.json"))
print(labels)

train = pd.read_csv('/content/drive/MyDrive/Python 공부/kaggle/train.csv')
train.head()

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [19]:
X_Train, Y_Train = train['image_id'].values, train['label'].values
X_Test = [name for name in (os.listdir(TEST_DIR))]

In [20]:
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:            
            return self.transform(x), self.lbs[index]            
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

In [21]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [22]:
trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=4)

testset = GetData(TEST_DIR, X_Test, None, Transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

In [26]:
trainset

In [29]:
x, y = next(iter(trainset))
print("tranform =", x)
print("\fname =", y)
print("\nsizes of x =")
#for key, value in x.items():
#   print(f"\t{key} = {value.size()}")

tranform = tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.804

In [30]:
x, y = next(iter(trainloader))
print("tranform =", x)
print("\fname =", y)
print("\nsizes of x =")
#for key, value in x.items():
#   print(f"\t{key} = {value.size()}")

tranform = tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044,

In [23]:
model = torchvision.models.resnet152()
model.fc = nn.Linear(2048, 5, bias=True)

In [24]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [25]:
for epoch in range(EPOCHS):
    tr_loss = 0.0

    model = model.train()

    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.detach().item()
    
    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss / i))

FileNotFoundError: ignored

In [ ]:
s_ls = []

with torch.no_grad():
    model.eval()
    for image, fname in testloader: 
        image = image.to(DEVICE)
        
        logits = model(image)        
        ps = torch.exp(logits)        
        _, top_class = ps.topk(1, dim=1)
        
        for pred in top_class:
            s_ls.append([fname[0], pred.item()])

In [ ]:
sub = pd.DataFrame.from_records(s_ls, columns=['image_id', 'label'])
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)